# Imports

In [1]:
import os
from pathlib import Path

import pandas as pd

In [2]:
if "notebooks" in os.getcwd():
    %cd ..

fn = Path("data/tyndp_2024_bundle/Offshore hubs/GENERATOR.xlsx")

/home/tgilon/Documents/oet/open-tyndp-ter


# Read potentials

In [3]:
df_layer_pot = (
    pd.read_excel(fn, sheet_name="LAYER_POTENTIAL")
    .groupby(by=["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO"])[
        ["EXISTING_MW", "MARGIN_MW", "MAX_MW"]
    ]
    .sum()
)
df_layer_pot

EXISTING_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
ALOR001       Radial             2030 Distributed Energy       0.0000   
                                      Global Ambition          0.0000   
                                      National Trends          0.0000   
                                 2035 Distributed Energy       0.0000   
                                      Global Ambition          0.0000   
...                                                               ...   
UKOR001       Radial             2045 Global Ambition      45240.6181   
                                      National Trends      45240.6181   
                                 2050 Distributed Energy   45990.6181   
                                      Global Ambition      45990.6181   
                                      National Trends      45990.6181   

                                                           MARGIN_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                         
ALOR001       Radial             2030 Distributed Energy     0.00000   
                                      Global Ambition        0.00000   
                                      National Trends        0.00000   
                                 2035 Distributed Energy     0.00000   
                                      Global Ambition        0.00000   
...                                                              ...   
UKOR001       Radial             2045 Global Ambition     2172.46668   
                                      National Trends     2172.46668   
                                 2050 Distributed Energy  1422.46668   
                                      Global Ambition     1422.46668   
                                      National Trends     1422.46668   

                                                               MAX_MW  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                         
ALOR001       Radial             2030 Distributed Energy      0.00000  
                                      Global Ambition         0.00000  
                                      National Trends         0.00000  
                                 2035 Distributed Energy      0.00000  
                                      Global Ambition         0.00000  
...                                                               ...  
UKOR001       Radial             2045 Global Ambition     47413.08478  
                                      National Trends     47413.08478  
                                 2050 Distributed Energy  47413.08478  
                                      Global Ambition     47413.08478  
                                      National Trends     47413.08478  

[840 rows x 3 columns]

In [4]:
df_zone_pot = pd.read_excel(fn, sheet_name="ZONE_POTENTIAL").set_index(
    ["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO"]
)[["EXISTING_MW", "MARGIN_MW", "MAX_MW"]]
df_zone_pot

EXISTING_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
ALOR001       Radial             2030 Distributed Energy         0.00   
                                 2035 Distributed Energy         0.00   
                                 2040 Distributed Energy         0.00   
                                 2045 Distributed Energy         0.00   
                                 2050 Distributed Energy         0.00   
...                                                               ...   
DEOR001       Radial             2030 National Trends         7548.75   
                                 2035 National Trends         7548.75   
                                 2040 National Trends         7486.75   
                                 2045 National Trends         3660.75   
                                 2050 National Trends         2699.75   

                                                          MARGIN_MW   MAX_MW  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                                
ALOR001       Radial             2030 Distributed Energy        0.0     0.00  
                                 2035 Distributed Energy        0.0     0.00  
                                 2040 Distributed Energy        0.0     0.00  
                                 2045 Distributed Energy        0.0     0.00  
                                 2050 Distributed Energy        0.0     0.00  
...                                                             ...      ...  
DEOR001       Radial             2030 National Trends           0.0  7548.75  
                                 2035 National Trends           0.0  7548.75  
                                 2040 National Trends          62.0  7548.75  
                                 2045 National Trends        3888.0  7548.75  
                                 2050 National Trends        4849.0  7548.75  

[840 rows x 3 columns]

# Compare

In [5]:
tol = 1e-3
df = (
    df_layer_pot.join(df_zone_pot, how="outer", rsuffix="2")
    .assign(
        EXISTING_MW_diff=lambda x: x.EXISTING_MW - x.EXISTING_MW2,
        MARGIN_MW_diff=lambda x: x.MARGIN_MW - x.MARGIN_MW2,
        MAX_MW_diff=lambda x: x.MAX_MW - x.MAX_MW2,
    )
    .query(
        "abs(EXISTING_MW_diff) > @tol or "
        "abs(MARGIN_MW_diff) > @tol or "
        "abs(MAX_MW_diff) > @tol"
    )
    .sort_values(by="MAX_MW_diff", ascending=False)[
        [
            "EXISTING_MW",
            "EXISTING_MW2",
            "EXISTING_MW_diff",
            "MARGIN_MW",
            "MARGIN_MW2",
            "MARGIN_MW_diff",
            "MAX_MW",
            "MAX_MW2",
            "MAX_MW_diff",
        ]
    ]
)
df

EXISTING_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
FIOH001       Hub                2030 Global Ambition         1000.00   
                                      National Trends         1000.00   
                                      Distributed Energy      1000.00   
DEOH001       FarShoreHub        2030 Global Ambition        18868.00   
FIOH001       Hub                2035 National Trends         3600.00   
...                                                               ...   
DEOH002       Hub                2045 National Trends         5828.55   
                                 2050 Distributed Energy      5105.30   
                                      Global Ambition         5105.30   
                                      National Trends         5105.30   
                                 2045 Distributed Energy      5828.55   

                                                          EXISTING_MW2  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                           
FIOH001       Hub                2030 Global Ambition          1000.00   
                                      National Trends          1000.00   
                                      Distributed Energy       1000.00   
DEOH001       FarShoreHub        2030 Global Ambition         18868.00   
FIOH001       Hub                2035 National Trends          3600.00   
...                                                                ...   
DEOH002       Hub                2045 National Trends          6354.55   
                                 2050 Distributed Energy       5105.30   
                                      Global Ambition          5105.30   
                                      National Trends          5105.30   
                                 2045 Distributed Energy       6354.55   

                                                          EXISTING_MW_diff  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                               
FIOH001       Hub                2030 Global Ambition                  0.0   
                                      National Trends                  0.0   
                                      Distributed Energy               0.0   
DEOH001       FarShoreHub        2030 Global Ambition                  0.0   
FIOH001       Hub                2035 National Trends                  0.0   
...                                                                    ...   
DEOH002       Hub                2045 National Trends               -526.0   
                                 2050 Distributed Energy               0.0   
                                      Global Ambition                  0.0   
                                      National Trends                  0.0   
                                 2045 Distributed Energy            -526.0   

                                                          MARGIN_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                        
FIOH001       Hub                2030 Global Ambition      64000.00   
                                      National Trends      64000.00   
                                      Distributed Energy   64000.00   
DEOH001       FarShoreHub        2030 Global Ambition      44000.00   
FIOH001       Hub                2035 National Trends      61400.00   
...                                                             ...   
DEOH002       Hub                2045 National Trends          0.00   
                                 2050 Distributed Energy     723.25   
                                      Global Ambition        723.25   
                                      National Trends        723.25   
                                 2045 Distributed Energy       0.00   

                                                          MARGIN_MW2  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                         
FIOH001       Hub                2030 Global Ambition        6

In [6]:
diffs = [c for c in df.columns if "diff" in c]
df[diffs].head(10)

EXISTING_MW_diff  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                               
FIOH001       Hub                2030 Global Ambition                  0.0   
                                      National Trends                  0.0   
                                      Distributed Energy               0.0   
DEOH001       FarShoreHub        2030 Global Ambition                  0.0   
FIOH001       Hub                2035 National Trends                  0.0   
DEOH001       FarShoreHub        2030 National Trends                  0.0   
                                      Distributed Energy               0.0   
FIOH001       Hub                2035 Global Ambition                  0.0   
                                      Distributed Energy               0.0   
FROH002       Hub                2030 National Trends                  0.0   

                                                          MARGIN_MW_diff  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                             
FIOH001       Hub                2030 Global Ambition       58000.000000   
                                      National Trends       58000.000000   
                                      Distributed Energy    58000.000000   
DEOH001       FarShoreHub        2030 Global Ambition       44000.000000   
FIOH001       Hub                2035 National Trends       44000.000000   
DEOH001       FarShoreHub        2030 National Trends       44000.000000   
                                      Distributed Energy    44000.000000   
FIOH001       Hub                2035 Global Ambition       44000.000000   
                                      Distributed Energy    44000.000000   
FROH002       Hub                2030 National Trends       36781.170944   

                                                           MAX_MW_diff  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
FIOH001       Hub                2030 Global Ambition     58000.000000  
                                      National Trends     58000.000000  
                                      Distributed Energy  58000.000000  
DEOH001       FarShoreHub        2030 Global Ambition     44000.000000  
FIOH001       Hub                2035 National Trends     44000.000000  
DEOH001       FarShoreHub        2030 National Trends     44000.000000  
                                      Distributed Energy  44000.000000  
FIOH001       Hub                2035 Global Ambition     44000.000000  
                                      Distributed Energy  44000.000000  
FROH002       Hub                2030 National Trends     36781.170944

In [8]:
df[diffs].tail(10)

EXISTING_MW_diff  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                               
NOSOR01       Radial             2045 Global Ambition                  0.0   
UKOH003       Hub                2045 National Trends                  0.0   
                                      Distributed Energy               0.0   
                                      Global Ambition                  0.0   
DEOH002       Hub                2045 Global Ambition               -526.0   
                                      National Trends               -526.0   
                                 2050 Distributed Energy               0.0   
                                      Global Ambition                  0.0   
                                      National Trends                  0.0   
                                 2045 Distributed Energy            -526.0   

                                                          MARGIN_MW_diff  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                             
NOSOR01       Radial             2045 Global Ambition         136.730240   
UKOH003       Hub                2045 National Trends          87.176512   
                                      Distributed Energy       87.176512   
                                      Global Ambition          87.176512   
DEOH002       Hub                2045 Global Ambition           0.000000   
                                      National Trends           0.000000   
                                 2050 Distributed Energy     -526.000000   
                                      Global Ambition        -526.000000   
                                      National Trends        -526.000000   
                                 2045 Distributed Energy        0.000000   

                                                          MAX_MW_diff  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                         
NOSOR01       Radial             2045 Global Ambition      136.730240  
UKOH003       Hub                2045 National Trends       87.176512  
                                      Distributed Energy    87.176512  
                                      Global Ambition       87.176512  
DEOH002       Hub                2045 Global Ambition     -526.000000  
                                      National Trends     -526.000000  
                                 2050 Distributed Energy  -526.000000  
                                      Global Ambition     -526.000000  
                                      National Trends     -526.000000  
                                 2045 Distributed Energy  -526.000000